In [1]:
import importlib
import sys
sys.path.append('../')
import torch
import torch.nn as nn
import torch.optim as optim
from Code.envs.MountainCar import MultiMountainCar, LookupPolicy, PassiveEnv
from Code.BNN import RBNN, SuperSpike, FlipFlopSpike, newRBNN


In [2]:
env = PassiveEnv()

In [3]:
BATCH_SIZE = 64

In [4]:
#model = RBNN(1, 64, 32, 512, 1, SuperSpike.apply, FlipFlopSpike.apply)
model = newRBNN(1, 64, 32, 64, 1, SuperSpike.apply, FlipFlopSpike.apply)


teacher = LookupPolicy()


#TODO: test superspike instead of Bellec

In [5]:
spikes = torch.ones([130])
def logger(h, t):
    spikes[t] = h['spikes'][0].sum()
        #spikes[t//10, 1, i] = h['spikes'][0][32:].sum()

        #print(t, i)
        #print(h['spikes'][0].sum())
    

In [6]:
#torch.autograd.set_detect_anomaly(True)

In [7]:
#bce = nn.BCELoss(reduction='none') #reduction='sum'
optimizer = optim.Adam(model.parameters(), lr=1e-4)#0.000011e-6




In [8]:
mse = nn.MSELoss(reduction='none')
inputs, targets, mask = env.getBatch(BATCH_SIZE)
model(inputs/0.4, h=None, logger=logger)
print(spikes)
for i in range(5000):
    model.zero_grad()
    inputs, targets, mask = env.getBatch(BATCH_SIZE)
    if i%100 == 0:
        # torch.autograd.set_detect_anomaly(False)
        for p in model.parameters():
            if torch.isnan(p).any():
                raise Exception('Corrupted Model')
    outputs, _ = model(inputs/0.4)
    #print(outputs.shape, targets.shape)
    loss = (mse(outputs.squeeze(dim=2)/50, targets) * mask).sum() / mask.sum()
    loss.backward()
    optimizer.step()
    #if i%100 == 0:
    #    torch.autograd.set_detect_anomaly(False)
    if i%10 == 0:
        print(loss.item(), (loss/targets.view(-1).var()).item(), i) #, ((outputs>0.5) != targets).sum()
    if i%50 == 0:
        model(inputs/0.4, h=None, logger=logger)
        print(spikes)


tensor([16., 18., 17., 17., 19., 20., 19., 19., 19., 19., 18., 16., 17., 16.,
        17., 16., 16., 15., 16., 14., 14., 15., 16., 15., 15., 16., 16., 16.,
        15., 14., 14., 14., 14., 15., 16., 15., 15., 15., 15., 15., 16., 15.,
        15., 15., 15., 16., 15., 15., 14., 15., 15., 15., 16., 16., 14., 14.,
        15., 15., 16., 15., 15., 16., 15., 15., 16., 15., 15., 17., 16., 17.,
        17., 17., 15., 15., 15., 15., 15., 15., 16., 15., 15., 16., 16., 15.,
        15., 15., 14., 15., 14., 14., 15., 13., 14., 14., 15., 15., 14., 13.,
        13., 13., 14., 14., 15., 16., 16., 17., 17., 16., 17., 16., 17., 16.,
        17., 16.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.], grad_fn=<CopySlices>)
0.0004768312501255423 0.9434186816215515 0
tensor([16., 16., 17., 16., 15., 16., 17., 17., 17., 17., 17., 17., 17., 18.,
        17., 17., 16., 16., 17., 17., 16., 17., 17., 16., 17., 17., 17., 15.,
        16., 15., 15., 15., 15., 15., 15., 15., 

In [9]:
model.adaptive_linear.weight

AttributeError: 'newRBNN' object has no attribute 'adaptive_linear'

In [ ]:
#torch.save(model, '../models/lstm_passive')
#0.0002

In [ ]:
#torch.save(model, '../models/super_RBNN2')
#0.001

In [ ]:
from matplotlib import pyplot as plt
#model = torch.load('../models/snn_passive3')
%matplotlib



In [ ]:
def doplot():
    inputs, targets, mask = env.getBatch(1)
    outputs, _ = model(inputs/0.4)
    plt.close()
    plt.plot(inputs[:, 0, 0], targets)
    plt.plot(inputs[:, 0, 0], outputs.squeeze().detach()/50)

In [ ]:
doplot()

In [ ]:

for p in model.parameters():
    print(p.shape)

In [ ]:
mse = nn.MSELoss(reduction='none')
#model(inputs/0.4)[0].squeeze()
model.zero_grad()
inputs, targets, mask = env.getBatch(1)
outputs, _ = model(inputs/0.4)
loss = (mse(outputs.squeeze(dim=2)/30, targets) * mask).sum() / mask.sum()
loss.backward()

In [ ]:
LR = 1e-4 / 2
opt1 = optim.Adam(model.getMemoryParameters(), lr=LR/5)
opt2 = optim.Adam(model.getOutputParameters(), lr=LR)

In [ ]:
mse = nn.MSELoss(reduction='none')
train_memory = True
#opt = opt2
for i in range(5000):
    model.zero_grad()
    inputs, targets, mask = env.getBatch(BATCH_SIZE)
    if i%100 == 0:
        train_memory = not train_memory
        print('Train Memory: ', train_memory)
        opt = opt1 if train_memory else opt2
        for p in model.parameters():
            if torch.isnan(p).any():
                raise Exception('Corrupted Model')
    outputs, _ = model(inputs/0.4)
    #print(outputs.shape, targets.shape)
    loss = (mse(outputs.squeeze(dim=2)/10, targets) * mask).sum() / mask.sum()
    loss.backward()
    opt.step()
    if i%10 == 0:
        print(loss.item(), (loss/targets.view(-1).var()).item(), i) #, ((outputs>0.5) != targets).sum()

In [ ]:
spikes = torch.ones([32, 130]) * -1
def allspikes(h, t):
    spikes[:, t] = h['spikes'][0]   

inputs, targets, mask = env.getBatch(1)
model(inputs/0.4, h=None, logger=allspikes)
torch.set_printoptions(profile="full")
print(spikes)